# 2021_05_03 Delay Analysis

Notebook to perform analysis on delay period (2 or 4 seconds) of PWM task recording from mPFC. Functions tested & debugged in event_centering.ipynb and moving here for clean start

### Libraries & Settings

In [1]:
# libraries
import sys; sys.path.insert(0, '..') # if you don't find it here, look one above
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.io as spio
import pickle
import warnings

from tqdm import tqdm
from scipy import stats
from scipy.ndimage import gaussian_filter1d
from io_utils import *
from plotting_utils import * 


# settings
%load_ext autoreload
%autoreload 2
sns.set_context("talk")

### Set paths & defaults

In [2]:
### Set paths & defaults

sess_names = np.array(['data_sdb_20190724_193007_fromSD', 'data_sdc_20190902_145404_fromSD', 'W122_07_27_2019_1_fromSD', 
            'W122_08_06_2019_1_fromSD', 'W122_08_07_2019_1_fromSD', 'W122_08_31_2019_1_fromSD', 'W122_09_04_2019_1_fromSD',
            'W122_09_09_2019_1_fromSD', 'W122_09_12_2019_1_fromSD','W122_09_21_2019_1_fromSD', 'W122_09_24_2019_1_fromSD'])

base_path  = 'Y:\jbreda\ephys\post_sort_analysis\sorted_pre_bdata' 
beh_mat   = 'protocol_info.mat'
spks_mat  = 'ksphy_clusters_foranalysis.mat'


# create paths
sess_paths = [os.path.join(base_path, sess) for sess in sess_names]
beh_paths  = [os.path.join(path, beh_mat) for path in sess_paths]
spks_paths = [os.path.join(path, spks_mat) for path in sess_paths]

# alginment info (hard coded here & in fx)
names = ['aud1on', 'aud1off', 'aud2on', 'aud2off', 'trial_all', 'delay_overlap', 
         'delay2s', 'trial2s', 'delay4s', 'trial4s']
windows = [[-400, 600], [-500, 500], [-400, 600], [-500, 500], [-100, 5000], [-300, 2300],
           [-150, 2150],[-200, 2400], [-150, 4150], [-200, 4400]]

align_windows = dict(zip(names,windows))

In [3]:
align_windows

{'aud1on': [-400, 600],
 'aud1off': [-500, 500],
 'aud2on': [-400, 600],
 'aud2off': [-500, 500],
 'trial_all': [-100, 5000],
 'delay_overlap': [-300, 2300],
 'delay2s': [-150, 2150],
 'trial2s': [-200, 2400],
 'delay4s': [-150, 4150],
 'trial4s': [-200, 4400]}

### Load in data & align spikes to events - 2 and 4 second delay period

I'm filtering the dataframe below specifically for delay analysis (no psychometric sounds, no violations, no 6 second trials)

In [20]:
events = ['delay2s', 'delay4s']
fig_save_path = os.path.join(os.getcwd(), 'figures', 'delay_loudness', '2s_4s_separate')

for name, path, behp, spksp in zip(sess_names, sess_paths, beh_paths, spks_paths):
    
    print(f"Loading session {name}")

    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] != 'viol')]
    
    del_2_df = delay_analysis_df[(delay_analysis_df['delay'] == 2)] 
    del_4_df = delay_analysis_df[(delay_analysis_df['delay'] == 4)]
    dfs = [del_2_df, del_4_df]
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=False, 
                                   file_name='delay_analysis_aligned_spks.pkl')
    
    print("Plotting")
    
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

Loading session data_sdb_20190724_193007_fromSD
Aligning to events
loading from file...
Plotting


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:176: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)


IndexError: index 42 is out of bounds for axis 0 with size 41

### Load in data & align spikes to events- overlapping 2s of both delay periods

The figures above appeared to have no affect of first sound loudness on firing rate. In going back to the fiugres from last meeting, I realized I was only doing hit trials, and I collapsed the 2 and 4 s trials together and only looked at the first 2000 ms of delay. So, I will try to repeate this.

In [4]:
events = ['delay_overlap']

for name, path, behp, spksp in zip(sess_names, sess_paths, beh_paths, spks_paths):
    
    print(f"Loading session {name}")
  
    beh_df, spks_dict = load_and_wrangle(behp, spksp, overwrite=False)
    
    delay_analysis_df = beh_df[(beh_df['delay'] != 6) & (beh_df['first_sound'] != 'psycho') &
                                     (beh_df['hit_hist'] == 'hit')]
    
    dfs = [delay_analysis_df] # keeping the same structure as above
    
    print("Aligning to events")
    
    sess_aligned = event_align_session(spks_dict, delay_analysis_df, path, overwrite=False, 
                                   file_name='delay_analysis_aligned_spks.pkl')
    
    print("Plotting")
    
    # Note, switched the window here for regression to be [300:-300] becuase of how
    # alignmnet was performed (currently line 399 in plotting utils)
    run_delay_analysis_for_session(name, sess_aligned, align_windows, events, 
                                   dfs, fig_save_path, path)
    

Loading session data_sdb_20190724_193007_fromSD
Aligning to events
loading from file...
Plotting


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:176: RuntimeWarning: Mean of empty slice
  smoothed_mean = np.nanmean(np.array(smoothed_trials), axis=0)
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:294: RuntimeWarning: Mean of empty slice
  counted_mean = np.nanmean(counted_spks, axis = 0)
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:357: RuntimeWarning: All-NaN axis encountered
  y_min = (1 - scale) * np.nanmin([np.min(mean[cond_id]) for cond_id in mean.keys()])
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:358: RuntimeWarning: All-NaN axis encountered
  y_max = (1 - scale) * np.nanmax([np.max(mean[cond_id]) for cond_id in mean.keys()])


for 0.0001* there are 2 for this session
for 0.0027 there are 20 for this session
for 0.0073 there are 26 for this session
for 0.0197 there are 33 for this session
for 0.0531* there are 13 for this session


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:357: RuntimeWarning: All-NaN axis encountered
  y_min = (1 - scale) * np.nanmin([np.min(mean[cond_id]) for cond_id in mean.keys()])
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:358: RuntimeWarning: All-NaN axis encountered
  y_max = (1 - scale) * np.nanmax([np.max(mean[cond_id]) for cond_id in mean.keys()])
C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:294: RuntimeWarning: Mean of empty slice
  counted_mean = np.nanmean(counted_spks, axis = 0)


for 0.0001* there are 2 for this session
for 0.0027 there are 20 for this session
for 0.0073 there are 26 for this session
for 0.0197 there are 33 for this session
for 0.0531* there are 13 for this session


...

for 0.0001* there are 11 for this session
for 0.0027 there are 32 for this session
for 0.0073 there are 47 for this session
for 0.0197 there are 49 for this session
for 0.0531* there are 20 for this session
for 0.0001* there are 11 for this session
for 0.0027 there are 32 for this session
for 0.0073 there are 47 for this session
for 0.0197 there are 49 for this session
for 0.0531* there are 20 for this session


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:294: RuntimeWarning: Mean of empty slice
  counted_mean = np.nanmean(counted_spks, axis = 0)


for 0.0001* there are 11 for this session
for 0.0027 there are 32 for this session
for 0.0073 there are 47 for this session
for 0.0197 there are 49 for this session
for 0.0531* there are 20 for this session


C:\Users\JB\github\jbreda_PWM_ephys_analysis\plotting_utils.py:294: RuntimeWarning: Mean of empty slice
  counted_mean = np.nanmean(counted_spks, axis = 0)


for 0.0001* there are 11 for this session
for 0.0027 there are 32 for this session
for 0.0073 there are 47 for this session
for 0.0197 there are 49 for this session
for 0.0531* there are 20 for this session
for 0.0001* there are 11 for this session
for 0.0027 there are 32 for this session
for 0.0073 there are 47 for this session
for 0.0197 there are 49 for this session
for 0.0531* there are 20 for this session
Loading session W122_09_24_2019_1_fromSD
Aligning to events
loading from file...
Plotting
for 0.0001* there are 13 for this session
for 0.0027 there are 27 for this session
for 0.0073 there are 37 for this session
for 0.0197 there are 42 for this session
for 0.0531* there are 24 for this session
for 0.0001* there are 13 for this session
for 0.0027 there are 27 for this session
for 0.0073 there are 37 for this session
for 0.0197 there are 42 for this session
for 0.0531* there are 24 for this session
